In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from catboost import EFstrType
#from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
import toad
import seaborn as sns

In [61]:
df_0 = pd.read_csv("Loan default data from 2007 to 2017/accepted_2007_to_2018q4.csv/accepted_2007_to_2018q4.csv")
df_0 = df_0.iloc[-100000:]
df_0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
2160701,95341991,NaN,4200.0,4200.0,4200.0,36 months,13.99,143.53,C,C3,NaN,NaN,MORTGAGE,50000.0,Not Verified,Dec-2016,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,other,Other,403xx,KY,14.88,1.0,Dec-1998,730.0,734.0,1.0,17.0,NaN,15.0,0.0,8629.0,26.9,27.0,w,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,Mar-2019,143.53,Apr-2019,Mar-2019,709.0,705.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,17.0,4.0,17.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2160702,95915413,NaN,15000.0,15000.0,15000.0,36 months,5.32,451.73,A,A1,Business Manager,10+ years,MORTGAGE,105000.0,Verified,Dec-2016,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,278xx,NC,14.24,0.0,Jun-1999,775.0,779.0,0.0,NaN,NaN,10.0,0.0,13367.0,38.9,20.0,w,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,Apr-2018,9518.51,NaN,Mar-2019,719.0,715.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,NaN,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,NaN,NaN,NaN,

In [62]:
df_0["loan_status"].value_counts()

Current               42338
Fully Paid            42137
Charged Off           13303
Late (31-120 days)     1376
In Grace Period         558
Late (16-30 days)       280
Default                   6
Name: loan_status, dtype: int64

### fully Paid 认为是0,charged of 和Late(16-30)认为是1，其它舍弃

In [63]:
def target(x):

    if x["loan_status"] == "Fully Paid":
        x["target"] = 0
    elif x["loan_status"] == "Charged off" or "Late (16-30 days)":
        x["target"] = 1
    else:
        x["target"] = 2
    return x

In [64]:
target(df_0.iloc[2])

<ipython-input-63-91fdd0a546a8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["target"] = 1
C:\Users\myt\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


id                       94378012
member_id                     NaN
loan_amnt                    4800
funded_amnt                  4800
funded_amnt_inv              4800
                           ...   
settlement_date               NaN
settlement_amount             NaN
settlement_percentage         NaN
settlement_term               NaN
target                          1
Name: 2160703, Length: 152, dtype: object

In [65]:
df_0 = df_0.apply(lambda x:target(x),axis =1)
#

In [66]:
del df_0["loan_status"]

### 删除单一属性的值

In [67]:
orig_columns = df_0.columns

drop_columns = []

for col in orig_columns:
    col_series = df_0[col].dropna().unique()  #去重唯一的属性
    if len(col_series) == 1:  #如果该特征的属性只有一个属性，就给过滤掉该特征
        drop_columns.append(col)

In [68]:
drop_columns

['desc',
 'policy_code',
 'verification_status_joint',
 'hardship_type',
 'deferral_term',
 'hardship_length']

In [69]:
df_0 = df_0.drop(drop_columns,axis = 1)

### 删除缺失率大于50%的列

In [70]:
half_count = df_0.shape[0]/2
df_0 = df_0.dropna(thresh=half_count,axis = 1)

In [71]:
df_0

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,target
2160701,95341991,4200.0,4200.0,4200.0,36 months,13.99,143.53,C,C3,NaN,NaN,MORTGAGE,50000.0,Not Verified,Dec-2016,n,https://lendingclub.com/browse/loanDetail.acti...,other,Other,403xx,KY,14.88,1.0,Dec-1998,730.0,734.0,1.0,17.0,15.0,0.0,8629.0,26.9,27.0,w,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,Mar-2019,143.53,Mar-2019,709.0,705.0,0.0,Individual,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,4.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,N,Cash,N,1
2160702,95915413,15000.0,15000.0,15000.0,36 months,5.32,451.73,A,A1,Business Manager,10+ years,MORTGAGE,105000.0,Verified,Dec-2016,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,278xx,NC,14.24,0.0,Jun-1999,775.0,779.0,0.0,NaN,10.0,0.0,13367.0,38.9,20.0,w,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,Apr-2018,9518.51,Mar-2019,719.0,715.0,0.0,Individual,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,N,Cash,N,0
2160703,94378012,4800.0,4800.0,4800.0,36 months,7.99,150.40,A,A5,Sr.reliability engineer,10+ years,OWN,110000.0,Not Verified,Dec-2016,n,https://lendingclub.com/browse/loanDetail.acti...,home_improvement,Home improvement,060xx,CT,16.29,0.0,Oct-1992,790.0,794.0,2.0,NaN,11.0,0.0,46669.0,42.4,30.0,w,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,Jan-2019,150.40,Mar-2019,649.0,645.0,0.0,Individual,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.0,16.7,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.0,5.0,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,N,Cash,N,1
2160704,95132764,10000.0,10000.0,9750.0,36 months,7.49,311.02,A,A4,owner,10+ years,RENT,65000.0,Source Verified,Dec-2016,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,077xx,NJ,23.50,1.0,Aug-1998,695.0,699.0,1.0,13.0,11.0,0.0,5522.0,62.8,17.0,f,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,Mar-2019,311.02,Mar-2019,644.0,640.0,0.0,Individual,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,88

### 删除一些没用的列，譬如emp_title, id 之类的

In [72]:
df_0 = df_0.drop(['id','url','title','emp_title','zip_code'],axis=1)

### 处理字符型数据

In [73]:
df_0["sub_grade"].value_counts()

C1    8340
B5    7919
B1    6746
C2    6592
C4    6580
B4    6516
C3    6247
C5    6205
B3    5158
B2    4999
D2    4423
D3    3169
A5    3141
D1    3041
A1    2751
D4    2467
A4    2347
D5    2144
A2    2068
A3    2059
E1    1719
E3    1159
E4     933
E5     781
E2     523
F1     430
F2     374
F3     274
F4     219
F5     213
G1     144
G2     110
G3      83
G4      69
G5      55
Name: sub_grade, dtype: int64

In [74]:
val_list = []
base_list = ["A","B","C","D","E","F","G"]
cvt_list= []
for i in range(7):
    for j in range(5):
        val_list.append(base_list[i]+str(j+1))
        cvt_list.append(i*10+j)
def sub_grade(x):
    try:
        x["sub_grade"] = cvt_list[val_list.index(x["sub_grade"])]
    except:
        x["sub_grade"] = 0
    return x
df_0 = df_0.apply(lambda x: sub_grade(x),axis = 1)
df_0

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,target
2160701,4200.0,4200.0,4200.0,36 months,13.99,143.53,C,22,NaN,MORTGAGE,50000.0,Not Verified,Dec-2016,n,other,KY,14.88,1.0,Dec-1998,730.0,734.0,1.0,17.0,15.0,0.0,8629.0,26.9,27.0,w,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,Mar-2019,143.53,Mar-2019,709.0,705.0,0.0,Individual,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,4.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,N,Cash,N,1
2160702,15000.0,15000.0,15000.0,36 months,5.32,451.73,A,0,10+ years,MORTGAGE,105000.0,Verified,Dec-2016,n,credit_card,NC,14.24,0.0,Jun-1999,775.0,779.0,0.0,NaN,10.0,0.0,13367.0,38.9,20.0,w,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,Apr-2018,9518.51,Mar-2019,719.0,715.0,0.0,Individual,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,N,Cash,N,0
2160703,4800.0,4800.0,4800.0,36 months,7.99,150.40,A,4,10+ years,OWN,110000.0,Not Verified,Dec-2016,n,home_improvement,CT,16.29,0.0,Oct-1992,790.0,794.0,2.0,NaN,11.0,0.0,46669.0,42.4,30.0,w,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,Jan-2019,150.40,Mar-2019,649.0,645.0,0.0,Individual,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.0,16.7,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.0,5.0,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,N,Cash,N,1
2160704,10000.0,10000.0,9750.0,36 months,7.49,311.02,A,3,10+ years,RENT,65000.0,Source Verified,Dec-2016,n,debt_consolidation,NJ,23.50,1.0,Aug-1998,695.0,699.0,1.0,13.0,11.0,0.0,5522.0,62.8,17.0,f,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,Mar-2019,311.02,Mar-2019,644.0,640.0,0.0,Individual,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.0,87.7,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.0,5.0,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.0,0.0,0.0,113211.0,73655.0,5000.0,104411.0,N,Cash,N,1
2160705,20000.0,20000.0,20000.0,36 months,10.49,649.96,B,11,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2016,n,credit_card,IL,23.88,0.0,May-1999,780.0,784.0,0.0,NaN,14.0,0.0,20218.0,16.9,30.0,w,5601.74,5601.7

In [75]:
df_0 = df_0.iloc[:-2] #删除最后两行

In [76]:
df_0["grade"].value_counts()

C    33964
B    31338
D    15244
A    12366
E     5115
F     1510
G      461
Name: grade, dtype: int64

In [77]:
def grade(x):
    base_list = ["A","B","C","D","E","F","G"]
    x["grade"] = base_list.index(x["grade"])
    return x
df_0 = df_0.apply(lambda x: grade(x),axis = 1)
df_0

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,target
2160701,4200.0,4200.0,4200.0,36 months,13.99,143.53,2,22,NaN,MORTGAGE,50000.0,Not Verified,Dec-2016,n,other,KY,14.88,1.0,Dec-1998,730.0,734.0,1.0,17.0,15.0,0.0,8629.0,26.9,27.0,w,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,Mar-2019,143.53,Mar-2019,709.0,705.0,0.0,Individual,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,4.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,N,Cash,N,1
2160702,15000.0,15000.0,15000.0,36 months,5.32,451.73,0,0,10+ years,MORTGAGE,105000.0,Verified,Dec-2016,n,credit_card,NC,14.24,0.0,Jun-1999,775.0,779.0,0.0,NaN,10.0,0.0,13367.0,38.9,20.0,w,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,Apr-2018,9518.51,Mar-2019,719.0,715.0,0.0,Individual,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,N,Cash,N,0
2160703,4800.0,4800.0,4800.0,36 months,7.99,150.40,0,4,10+ years,OWN,110000.0,Not Verified,Dec-2016,n,home_improvement,CT,16.29,0.0,Oct-1992,790.0,794.0,2.0,NaN,11.0,0.0,46669.0,42.4,30.0,w,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,Jan-2019,150.40,Mar-2019,649.0,645.0,0.0,Individual,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.0,16.7,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.0,5.0,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,N,Cash,N,1
2160704,10000.0,10000.0,9750.0,36 months,7.49,311.02,0,3,10+ years,RENT,65000.0,Source Verified,Dec-2016,n,debt_consolidation,NJ,23.50,1.0,Aug-1998,695.0,699.0,1.0,13.0,11.0,0.0,5522.0,62.8,17.0,f,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,Mar-2019,311.02,Mar-2019,644.0,640.0,0.0,Individual,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.0,87.7,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.0,5.0,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.0,0.0,0.0,113211.0,73655.0,5000.0,104411.0,N,Cash,N,1
2160705,20000.0,20000.0,20000.0,36 months,10.49,649.96,1,11,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2016,n,credit_card,IL,23.88,0.0,May-1999,780.0,784.0,0.0,NaN,14.0,0.0,20218.0,16.9,30.0,w,5601.74,5601.7

**label encoder**

In [78]:
df_0_copy = df_0.copy()

In [79]:
label_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        None: 0
    },
    "term":{
        "36 months": 36,
        "60 months": 60,
        None: 0
    },
    "debt_settlement_flag":{
        "N": 0,
        "Y": 1
    }
}
df_0 = df_0.replace(label_dict)

In [80]:
pd.set_option('display.max_columns', None) #显示所有列
df_0

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,target
2160701,4200.0,4200.0,4200.0,36 months,13.99,143.53,2,22,0,MORTGAGE,50000.0,Not Verified,Dec-2016,n,other,KY,14.88,1.0,Dec-1998,730.0,734.0,1.0,17.0,15.0,0.0,8629.0,26.9,27.0,w,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,Mar-2019,143.53,Mar-2019,709.0,705.0,0.0,Individual,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,4.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,N,Cash,0,1
2160702,15000.0,15000.0,15000.0,36 months,5.32,451.73,0,0,10,MORTGAGE,105000.0,Verified,Dec-2016,n,credit_card,NC,14.24,0.0,Jun-1999,775.0,779.0,0.0,NaN,10.0,0.0,13367.0,38.9,20.0,w,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,Apr-2018,9518.51,Mar-2019,719.0,715.0,0.0,Individual,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,N,Cash,0,0
2160703,4800.0,4800.0,4800.0,36 months,7.99,150.40,0,4,10,OWN,110000.0,Not Verified,Dec-2016,n,home_improvement,CT,16.29,0.0,Oct-1992,790.0,794.0,2.0,NaN,11.0,0.0,46669.0,42.4,30.0,w,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,Jan-2019,150.40,Mar-2019,649.0,645.0,0.0,Individual,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.0,16.7,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.0,5.0,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,N,Cash,0,1
2160704,10000.0,10000.0,9750.0,36 months,7.49,311.02,0,3,10,RENT,65000.0,Source Verified,Dec-2016,n,debt_consolidation,NJ,23.50,1.0,Aug-1998,695.0,699.0,1.0,13.0,11.0,0.0,5522.0,62.8,17.0,f,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,Mar-2019,311.02,Mar-2019,644.0,640.0,0.0,Individual,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.0,87.7,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.0,5.0,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.0,0.0,0.0,113211.0,73655.0,5000.0,104411.0,N,Cash,0,1
2160705,20000.0,20000.0,20000.0,36 months,10.49,649.96,1,11,10,MORTGAGE,110000.0,Source Verified,Dec-2016,n,credit_card,IL,23.88,0.0,May-1999,780.0,784.0,0.0,NaN,14.0,0.0,20218.0,16.9,30.0,w,5601.74,5601.74,17519.780000,17519.78,14398.

In [81]:
df_0["disbursement_method"].value_counts()

Cash         99961
DirectPay       37
Name: disbursement_method, dtype: int64

In [82]:
df_0["term"]

2160701     36 months
2160702     36 months
2160703     36 months
2160704     36 months
2160705     36 months
              ...    
2260694     60 months
2260695     60 months
2260696     60 months
2260697     60 months
2260698     60 months
Name: term, Length: 99998, dtype: object

In [83]:
df_0["term"] = df_0["term"].apply(lambda x: 36 if x=="36 months" else 60)

In [84]:
df_0["purpose"].value_counts()

debt_consolidation    57766
credit_card           19782
home_improvement       7156
other                  7129
major_purchase         2370
medical                1423
small_business         1170
car                    1110
vacation                801
moving                  739
house                   458
renewable_energy         93
wedding                   1
Name: purpose, dtype: int64

In [85]:
df_0["pymnt_plan"].value_counts() #字段含义不明，应该删除

n    99960
y       38
Name: pymnt_plan, dtype: int64

In [86]:
df_0["addr_state"].value_counts() #地域的，使用one-hot-encoding吧，预处理先不管了

CA    13168
TX     8536
NY     7990
FL     7489
IL     4135
NJ     3594
PA     3577
OH     3419
GA     3310
NC     2762
MI     2760
VA     2593
AZ     2455
MD     2417
MA     2212
CO     2101
MN     1861
IN     1809
WA     1807
TN     1688
MO     1623
CT     1580
NV     1465
WI     1313
SC     1260
AL     1253
LA     1143
KY     1032
OR     1011
OK      913
KS      817
AR      797
UT      666
MS      643
NM      520
NE      517
HI      475
NH      467
RI      463
DE      298
ME      293
ID      260
MT      245
ND      233
VT      222
DC      219
WY      208
SD      201
AK      178
Name: addr_state, dtype: int64

In [87]:
df_0["initial_list_status"].value_counts()
df_0["initial_list_status"] = df_0["initial_list_status"].apply(lambda x: 1 if x == "f" else 0)

In [88]:
df_0["application_type"].value_counts()
df_0["application_type"] = df_0["application_type"].apply(lambda x: 0 if x =="Individual" else 1)

In [89]:
del_list = ["pymnt_plan","earliest_cr_line","last_pymnt_d","last_pymnt_amnt","last_credit_pull_d","disbursement_method","issue_d"]

In [90]:
df_0 = df_0.drop(del_list,axis = 1)

In [91]:
df_0 #home_ownership,verification_status,purpose,hardship_flag

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
2160701,4200.0,4200.0,4200.0,60,13.99,143.53,2,22,0,MORTGAGE,50000.0,Not Verified,other,KY,14.88,1.0,730.0,734.0,1.0,17.0,15.0,0.0,8629.0,26.9,27.0,0,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,709.0,705.0,0.0,0,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,4.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,N,0,1
2160702,15000.0,15000.0,15000.0,60,5.32,451.73,0,0,10,MORTGAGE,105000.0,Verified,credit_card,NC,14.24,0.0,775.0,779.0,0.0,NaN,10.0,0.0,13367.0,38.9,20.0,0,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,719.0,715.0,0.0,0,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,N,0,0
2160703,4800.0,4800.0,4800.0,60,7.99,150.40,0,4,10,OWN,110000.0,Not Verified,home_improvement,CT,16.29,0.0,790.0,794.0,2.0,NaN,11.0,0.0,46669.0,42.4,30.0,0,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,649.0,645.0,0.0,0,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.0,16.7,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.0,5.0,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,N,0,1
2160704,10000.0,10000.0,9750.0,60,7.49,311.02,0,3,10,RENT,65000.0,Source Verified,debt_consolidation,NJ,23.50,1.0,695.0,699.0,1.0,13.0,11.0,0.0,5522.0,62.8,17.0,1,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,644.0,640.0,0.0,0,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.0,87.7,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.0,5.0,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.0,0.0,0.0,113211.0,73655.0,5000.0,104411.0,N,0,1
2160705,20000.0,20000.0,20000.0,60,10.49,649.96,1,11,10,MORTGAGE,110000.0,Source Verified,credit_card,IL,23.88,0.0,780.0,784.0,0.0,NaN,14.0,0.0,20218.0,16.9,30.0,0,5601.74,5601.74,17519.780000,17519.78,14398.26,3121.52,0.00,0.0,0.0,744.0,740.0,0.0,0,0.0,0.0,202964.0,2.0,3.0,1.0,2.0,2.0,50453.0,65.0,1.0,1.0,6865.0,36.0,119500.0,0.0,0.0,3.0,3.0,14497.0,88311.0,18.6,0.0,0.0,142.0,211.0,4.0,2.0,2.0,4.0,2.0,0.0,5.0,6.0,6.0,9.0,7.0,10.0,21.0,6.0,14.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,348565.0,70671.0,108500.0,77565.0,N,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [92]:
df_0["home_ownership"].value_counts()

MORTGAGE    49025
RENT        39104
OWN         11766
ANY           103
Name: home_ownership, dtype: int64

In [93]:
df_0["verification_status"].value_counts()

Source Verified    40816
Verified           29961
Not Verified       29221
Name: verification_status, dtype: int64

In [94]:
df_0["purpose"].value_counts()

debt_consolidation    57766
credit_card           19782
home_improvement       7156
other                  7129
major_purchase         2370
medical                1423
small_business         1170
car                    1110
vacation                801
moving                  739
house                   458
renewable_energy         93
wedding                   1
Name: purpose, dtype: int64

In [95]:
df_0["hardship_flag"].value_counts()

N    99945
Y       53
Name: hardship_flag, dtype: int64

In [96]:
label_dict = {
    "home_ownership":{
        "MORTGAGE": 0,
        "RENT": 1,
        "OWN": 2,
        "ANY": 3,
        None:-1
    },
    "verification_status":{
        "Source Verified":0,
        "Verified":1,
        "Not Verified":2,
        None: -1
    },
    "purpose":{
        "debt_consolidation":0,
        "credit_card":1,
        "home_improvement":2,
        "other":3,
        "major_purchase":4,
        "medical":5,
        "small_business":6,
        "car":7,
        "vacation":8,
        "moving":9,
        "house":10,
        "renewable_energy":11,
        "wedding":12
    },
    "hardship_flag":{
        "N": 0,
        "Y": 1
    }

}
df_0 = df_0.replace(label_dict)
df_0

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
2160701,4200.0,4200.0,4200.0,60,13.99,143.53,2,22,0,0,50000.0,2,3,KY,14.88,1.0,730.0,734.0,1.0,17.0,15.0,0.0,8629.0,26.9,27.0,0,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,709.0,705.0,0.0,0,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,4.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,0,0,1
2160702,15000.0,15000.0,15000.0,60,5.32,451.73,0,0,10,0,105000.0,1,1,NC,14.24,0.0,775.0,779.0,0.0,NaN,10.0,0.0,13367.0,38.9,20.0,0,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,719.0,715.0,0.0,0,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,0,0,0
2160703,4800.0,4800.0,4800.0,60,7.99,150.40,0,4,10,2,110000.0,2,2,CT,16.29,0.0,790.0,794.0,2.0,NaN,11.0,0.0,46669.0,42.4,30.0,0,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,649.0,645.0,0.0,0,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.0,16.7,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.0,5.0,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,0,0,1
2160704,10000.0,10000.0,9750.0,60,7.49,311.02,0,3,10,1,65000.0,0,0,NJ,23.50,1.0,695.0,699.0,1.0,13.0,11.0,0.0,5522.0,62.8,17.0,1,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,644.0,640.0,0.0,0,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.0,87.7,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.0,5.0,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.0,0.0,0.0,113211.0,73655.0,5000.0,104411.0,0,0,1
2160705,20000.0,20000.0,20000.0,60,10.49,649.96,1,11,10,0,110000.0,0,1,IL,23.88,0.0,780.0,784.0,0.0,NaN,14.0,0.0,20218.0,16.9,30.0,0,5601.74,5601.74,17519.780000,17519.78,14398.26,3121.52,0.00,0.0,0.0,744.0,740.0,0.0,0,0.0,0.0,202964.0,2.0,3.0,1.0,2.0,2.0,50453.0,65.0,1.0,1.0,6865.0,36.0,119500.0,0.0,0.0,3.0,3.0,14497.0,88311.0,18.6,0.0,0.0,142.0,211.0,4.0,2.0,2.0,4.0,2.0,0.0,5.0,6.0,6.0,9.0,7.0,10.0,21.0,6.0,14.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,348565.0,70671.0,108500.0,77565.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [97]:
df_0

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
2160701,4200.0,4200.0,4200.0,60,13.99,143.53,2,22,0,0,50000.0,2,3,KY,14.88,1.0,730.0,734.0,1.0,17.0,15.0,0.0,8629.0,26.9,27.0,0,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,709.0,705.0,0.0,0,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.0,45.3,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.0,4.0,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.7,0.0,0.0,160033.0,13460.0,15000.0,15500.0,0,0,1
2160702,15000.0,15000.0,15000.0,60,5.32,451.73,0,0,10,0,105000.0,1,1,NC,14.24,0.0,775.0,779.0,0.0,NaN,10.0,0.0,13367.0,38.9,20.0,0,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,719.0,715.0,0.0,0,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.0,47.1,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.0,NaN,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.0,0.0,0.0,155369.0,50544.0,28400.0,42806.0,0,0,0
2160703,4800.0,4800.0,4800.0,60,7.99,150.40,0,4,10,2,110000.0,2,2,CT,16.29,0.0,790.0,794.0,2.0,NaN,11.0,0.0,46669.0,42.4,30.0,0,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,649.0,645.0,0.0,0,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.0,16.7,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.0,5.0,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,509315.0,68405.0,85000.0,40715.0,0,0,1
2160704,10000.0,10000.0,9750.0,60,7.49,311.02,0,3,10,1,65000.0,0,0,NJ,23.50,1.0,695.0,699.0,1.0,13.0,11.0,0.0,5522.0,62.8,17.0,1,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,644.0,640.0,0.0,0,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.0,87.7,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.0,5.0,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.0,0.0,0.0,113211.0,73655.0,5000.0,104411.0,0,0,1
2160705,20000.0,20000.0,20000.0,60,10.49,649.96,1,11,10,0,110000.0,0,1,IL,23.88,0.0,780.0,784.0,0.0,NaN,14.0,0.0,20218.0,16.9,30.0,0,5601.74,5601.74,17519.780000,17519.78,14398.26,3121.52,0.00,0.0,0.0,744.0,740.0,0.0,0,0.0,0.0,202964.0,2.0,3.0,1.0,2.0,2.0,50453.0,65.0,1.0,1.0,6865.0,36.0,119500.0,0.0,0.0,3.0,3.0,14497.0,88311.0,18.6,0.0,0.0,142.0,211.0,4.0,2.0,2.0,4.0,2.0,0.0,5.0,6.0,6.0,9.0,7.0,10.0,21.0,6.0,14.0,0.0,0.0,0.0,2.0,100.0,0.0,0.0,0.0,348565.0,70671.0,108500.0,77565.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [124]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
city_list = pd.DataFrame(df_0["addr_state"].value_counts()).T.columns
le.fit(city_list)

LabelEncoder()

In [126]:
df_0["addr_state"] = le.transform(df_0["addr_state"])
# 解码 city_list_new = le.inverse_transform(city_list_le) 

**保存编码器**

In [131]:
import pickle
obj=pickle.dumps(le)
with open("encoder","ab")as f:
    f.write(obj)

**读取编码器**

In [134]:
'''
f=open("encoder","rb")
while 1:
    try:
        obj = pickle.load(f)
    except:
        break
f.close()
le = obj
'''

'\nf=open("encoder","rb")\nwhile 1:\n    try:\n        obj = pickle.load(f)\n    except:\n        break\nf.close()\nle = obj\n'

In [127]:
df_0

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,4200.0,4200.0,4200.0,60.0,13.99,143.53,2.0,22.0,0.0,0.0,50000.0,2.0,3.0,16,14.88,1.0,730.0,734.0,1.0,17.000000,15.0,0.0,8629.0,26.9,27.0,0.0,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,709.0,705.0,0.0,0.0,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.000000,45.300000,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.000000,4.00000,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.70000,0.0,0.0,160033.0,13460.0,15000.0,15500.0,0.0,0.0,1.0
1,15000.0,15000.0,15000.0,60.0,5.32,451.73,0.0,0.0,10.0,0.0,105000.0,1.0,1.0,26,14.24,0.0,775.0,779.0,0.0,33.642519,10.0,0.0,13367.0,38.9,20.0,0.0,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,719.0,715.0,0.0,0.0,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.000000,47.100000,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.000000,6.86811,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.00000,0.0,0.0,155369.0,50544.0,28400.0,42806.0,0.0,0.0,0.0
2,4800.0,4800.0,4800.0,60.0,7.99,150.40,0.0,4.0,10.0,2.0,110000.0,2.0,2.0,6,16.29,0.0,790.0,794.0,2.0,33.642519,11.0,0.0,46669.0,42.4,30.0,0.0,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,649.0,645.0,0.0,0.0,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.000000,16.700000,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.000000,5.00000,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.00000,0.0,0.0,509315.0,68405.0,85000.0,40715.0,0.0,0.0,1.0
3,10000.0,10000.0,9750.0,60.0,7.49,311.02,0.0,3.0,10.0,1.0,65000.0,0.0,0.0,30,23.50,1.0,695.0,699.0,1.0,13.000000,11.0,0.0,5522.0,62.8,17.0,1.0,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,644.0,640.0,0.0,0.0,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.000000,87.700000,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.000000,5.00000,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.00000,0.0,0.0,113211.0,73655.0,5000.0,104411.0,0.0,0.0,1.0
4,20000.0,20000.0,20000.0,60.0,10.49,649.96,1.0,11.0,10.0,0.0,110000.0,0.0,1.0,13,23.88,0.0,780.0,784.0,0.0,33.642519,14.0,0.0,20218.0,16.9,30.0,0.0,5601.74,5601.74,17519.780000,17519.78,14398.26,3121.52,0.00,0.0,0.0,744.0,740.0,0.0,0.0,0.0,0.0,202964.0,2.0,3.0,1.0,2.0,2.0,50453.0,65.0,1.0,1.0,6865.0,36.0,119500.0,0.0,0.0,3.0,3.0,14497.0,88311.000000,18.600000,0.0,0.0,142.0,211.0,4.0,2.0,2.0,4.000000,2.00000,0.0,5.0,6.0,6.0,9.0,7.0,10.0,21.0,6.0,14.0,0.0,0.0,0.0,2.0,100.0,0.00000,0.0,0.0,348565.0,70671.0,108500.0,77565.0,

## ↑转化为字符型完成，剩下处理一下空缺值

In [135]:
df_0.reset_index(drop = True, inplace = True)

In [136]:
imp_mean = SimpleImputer() #实例化,默认均值填补

for i in range(df_0.shape[1]):
    if df_0.columns[i] == "addr_state":
        continue
    col = df_0.iloc[:, i].values.reshape(-1, 1)
    new_col = imp_mean.fit_transform(col)
    df_0.iloc[:, i] = new_col
df_0

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,4200.0,4200.0,4200.0,60.0,13.99,143.53,2.0,22.0,0.0,0.0,50000.0,2.0,3.0,16,14.88,1.0,730.0,734.0,1.0,17.000000,15.0,0.0,8629.0,26.9,27.0,0.0,1219.41,1219.41,3872.050000,3872.05,2980.59,891.46,0.00,0.0,0.0,709.0,705.0,0.0,0.0,0.0,0.0,125662.0,1.0,2.0,0.0,2.0,13.0,4831.0,31.0,0.0,0.0,5881.0,28.0,32100.0,2.0,0.0,2.0,3.0,11424.0,8202.000000,45.300000,0.0,0.0,171.0,216.0,34.0,3.0,3.0,34.000000,4.00000,0.0,2.0,6.0,3.0,6.0,6.0,12.0,18.0,6.0,15.0,0.0,0.0,0.0,1.0,96.3,66.70000,0.0,0.0,160033.0,13460.0,15000.0,15500.0,0.0,0.0,1.0
1,15000.0,15000.0,15000.0,60.0,5.32,451.73,0.0,0.0,10.0,0.0,105000.0,1.0,1.0,26,14.24,0.0,775.0,779.0,0.0,33.642519,10.0,0.0,13367.0,38.9,20.0,0.0,0.00,0.00,15838.304347,15838.30,15000.00,838.30,0.00,0.0,0.0,719.0,715.0,0.0,0.0,0.0,0.0,60303.0,1.0,4.0,3.0,3.0,5.0,37177.0,87.0,0.0,1.0,7277.0,66.0,34400.0,0.0,0.0,0.0,4.0,6030.0,15033.000000,47.100000,0.0,0.0,142.0,210.0,22.0,5.0,2.0,22.000000,6.86811,0.0,3.0,3.0,3.0,4.0,11.0,5.0,7.0,3.0,10.0,0.0,0.0,0.0,3.0,100.0,0.00000,0.0,0.0,155369.0,50544.0,28400.0,42806.0,0.0,0.0,0.0
2,4800.0,4800.0,4800.0,60.0,7.99,150.40,0.0,4.0,10.0,2.0,110000.0,2.0,2.0,6,16.29,0.0,790.0,794.0,2.0,33.642519,11.0,0.0,46669.0,42.4,30.0,0.0,1729.75,1729.75,3607.470000,3607.47,3070.25,537.22,0.00,0.0,0.0,649.0,645.0,0.0,0.0,0.0,0.0,316154.0,0.0,3.0,0.0,2.0,17.0,21736.0,53.0,2.0,6.0,6676.0,45.0,119000.0,4.0,0.0,4.0,8.0,28741.0,72306.000000,16.700000,0.0,0.0,63.0,290.0,7.0,7.0,3.0,7.000000,5.00000,0.0,4.0,5.0,6.0,16.0,5.0,7.0,22.0,5.0,11.0,0.0,0.0,0.0,2.0,100.0,0.00000,0.0,0.0,509315.0,68405.0,85000.0,40715.0,0.0,0.0,1.0
3,10000.0,10000.0,9750.0,60.0,7.49,311.02,0.0,3.0,10.0,1.0,65000.0,0.0,0.0,30,23.50,1.0,695.0,699.0,1.0,13.000000,11.0,0.0,5522.0,62.8,17.0,1.0,2716.54,2648.63,8393.380000,8183.55,7283.46,1109.92,0.00,0.0,0.0,644.0,640.0,0.0,0.0,0.0,0.0,73655.0,1.0,6.0,0.0,0.0,29.0,68133.0,25.0,1.0,3.0,2512.0,31.0,8800.0,0.0,0.0,1.0,3.0,7366.0,616.000000,87.700000,0.0,0.0,184.0,220.0,5.0,5.0,0.0,5.000000,5.00000,0.0,2.0,4.0,2.0,3.0,6.0,5.0,11.0,4.0,10.0,0.0,0.0,0.0,1.0,88.2,100.00000,0.0,0.0,113211.0,73655.0,5000.0,104411.0,0.0,0.0,1.0
4,20000.0,20000.0,20000.0,60.0,10.49,649.96,1.0,11.0,10.0,0.0,110000.0,0.0,1.0,13,23.88,0.0,780.0,784.0,0.0,33.642519,14.0,0.0,20218.0,16.9,30.0,0.0,5601.74,5601.74,17519.780000,17519.78,14398.26,3121.52,0.00,0.0,0.0,744.0,740.0,0.0,0.0,0.0,0.0,202964.0,2.0,3.0,1.0,2.0,2.0,50453.0,65.0,1.0,1.0,6865.0,36.0,119500.0,0.0,0.0,3.0,3.0,14497.0,88311.000000,18.600000,0.0,0.0,142.0,211.0,4.0,2.0,2.0,4.000000,2.00000,0.0,5.0,6.0,6.0,9.0,7.0,10.0,21.0,6.0,14.0,0.0,0.0,0.0,2.0,100.0,0.00000,0.0,0.0,348565.0,70671.0,108500.0,77565.0,

## 建模过程

In [137]:
data = df_0.iloc[:,:-1]
target = df_0.iloc[:,-1]

In [138]:
from sklearn.model_selection import train_test_split
XTrain, XTest, YTrain, YTest = train_test_split(data, target, test_size=0.3,random_state = 42)

In [139]:
import xgboost as xgb
import catboost
ctb_clf = xgb.XGBClassifier(n_estimators=120,#500棵树  #90 0.42
    learning_rate =0.1,  
    max_depth=3,
    min_child_weight=1,
    gamma=0.3,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=12,
    scale_pos_weight=1,
    reg_lambda=1,
    seed=27)
#ctb_clf= catboost.CatBoostClassifier()
ctb_clf.fit(XTrain, YTrain, verbose=True)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:47:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.3, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=120, n_jobs=12, nthread=12, num_parallel_tree=1,
              random_state=27, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=27, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [142]:
ctb_clf.score(XTest,YTest)

0.9967666666666667

In [143]:
df_0.to_csv("processed_file.csv")